### The brain and CSF - a fluid-poroelastic interaction problem

In this notebook, we run the main finite element simulation of the brain-CSF fluid-poroelastic interaction problem.
First, we import both FEniCS and Multiphenics and the definition of function spaces, measures and the weak form of the system:

In [ ]:
from fenics import *
from multiphenics import *
from biotStokesWeakForm import (create_function_spaces, 
                                create_measures, biot_stokes_system)
import numpy as np
from tqdm.notebook import tqdm
import os

Next, we specify the relevant subdomain and facet ids, the simulation time, material parameter and the source term *g*, which represents the net blood flow into the brain tissue and drives the dynamcics: 

In [ ]:
porous_id = 1
fluid_id = 2

# facet ids
skull_id = 1
spinal_canal_id = 2
interface_id = 3
spinal_cord_id = 4
aqueduct_V4_id = 5

u_degree, p_degree = (2,1)
T = 8
num_steps = int(T*40)
dt = T / num_steps
times = np.linspace(0, T, num_steps + 1)
nu = 0.479
E = 2000
material_parameter = {"kappa":1e-14,"lmbda":nu * E / ((1.0 - 2.0 * nu) * (1.0 + nu)),
                      "c":1e-7,"mu_s":E / (2.0 * (1.0 + nu)),
                      "rho_s":1000, "alpha":1, "rho_f":1000, "mu_f":0.0007,
                      "gamma":1}
g = Expression("A*sin(2*M_PI*f*t)", f =1, t=0, A=0.01, degree=0)  #roughly 10ml/s peak per 1000ml volume
names = ["u", "pF", "d", "pP", "phi"]


Additionally, we read in the mesh with the subdomain marker, the facet marker and the restrictions required by Multiphenics:

In [ ]:
mesh = Mesh()
with XDMFFile("mesh/mesh.xdmf") as f:
    f.read(mesh)
    sm = MeshFunction("size_t", mesh, 3, 0)
    f.read(sm, "subdomains")
    
with XDMFFile("mesh/facets.xdmf") as f:
    bm = MeshFunction("size_t", mesh, 2, 0)
    f.read(bm)
    
fluid_restriction = MeshRestriction(mesh, "mesh/fluid.rtc.xdmf")
porous_restriction = MeshRestriction(mesh, "mesh/porous.rtc.xdmf")

Then, we use the imported functions to setup the measures, the block function space and the left- and right-hand-side of the system. 
Further, we specify the Dirichlet boundary conditions for the skull and the spinal cord and setup a *MUMPS* direct solver object.

In [ ]:
measures = create_measures(mesh, sm, bm,
                           fluid_id, porous_id, interface_id)

H = create_function_spaces(mesh, u_degree, p_degree,
                           fluid_restriction, porous_restriction)

lhs, rhs, block_function = biot_stokes_system(mesh, material_parameter,
                                              H, measures, dt, g_source=g) 
bc_d = DirichletBC(H.sub(2), Constant((0,0,0)), bm, spinal_cord_id)
bc_u = DirichletBC(H.sub(0), Constant((0,0,0)), bm, skull_id)
bcs = BlockDirichletBC([bc_d, bc_u])
AA = block_assemble(lhs, keep_diagonal=True)
bcs.apply(AA)
solver = PETScLUSolver(AA, "mumps")

Finally, we set up writing the results to file and start the main time stepping loop: We update the time of the source term, reassemble the right-hand-side and call the main *solve*.

In [ ]:
xdmf_vis = XDMFFile("results_vis.xdmf" )
xdmf_vis.parameters["functions_share_mesh"] = True
xdmf_vis.parameters["rewrite_function_mesh"] = False
xdmf_postp = XDMFFile("results_postp.xdmf" )

for k,f in enumerate(block_split(block_function)):
        f.rename(names[k], "")
        xdmf_postp.write_checkpoint(f,names[k], 0, xdmf_postp.Encoding.HDF5, False)
        xdmf_vis.write(f, 0)

for t in tqdm(times):
    g.t = t
    FF = block_assemble(rhs)
    bcs.apply(FF)
    solver.solve(block_function.block_vector(), FF)
    block_function.block_vector().block_function().apply("to subfunctions")
    results = block_split(block_function)
    for k,f in enumerate(results):
        f.rename(names[k], "")
        xdmf_postp.write_checkpoint(f, names[k], t, xdmf_postp.Encoding.HDF5, True)
        xdmf_vis.write(f,t)
xdmf_vis.close()
xdmf_postp.close()